In [3]:
import streamlit as st
import pandas as pd
import subprocess
import sys
import numpy as np
import requests
import time
import altair as alt
import pandas as pd
import streamlit as st
from vega_datasets import data
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options


2022-04-27 12:13:19.623 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [5]:
fa=pd.read_excel('fa.xlsx')
ooc=pd.read_excel('ooc.xlsx')
other=pd.read_excel('other.xlsx')
#display(fa)
#display(ooc)
#fa.fillna(0)
#ooc.fillna(0)
#display(other)

,0_Rk,1_Player,2_Nation,3_Pos,4_Squad,5_Comp,6_Age,7_Born,Playing Time_MP,Playing Time_Starts,...,Expected_xG,Expected_npxG,Expected_xA,Expected_npxG+xA,Per 90 Minutes_xG,Per 90 Minutes_xA,Per 90 Minutes_xG+xA,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xA,33_Matches
0,1,Max Aarons,eng ENG,DF,Norwich City,eng Premier League,22-113,2000,29,27,...,0.7,0.7,1.1,1.7,0.02,0.04,0.06,0.02,0.06,Matches
1,2,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34-211,1987,30,30,...,1.2,1.2,0.2,1.4,0.04,0.01,0.05,0.04,0.05,Matches
2,3,Salis Abdul Samed,gh GHA,MF,Clermont Foot,fr Ligue 1,22-032,2000,29,28,...,0.8,0.8,1.0,1.8,0.03,0.04,0.07,0.03,0.07,Matches
3,4,Laurent Abergel,fr FRA,MF,Lorient,fr Ligue 1,29-085,1993,30,30,...,1.9,1.9,2.2,4.1,0.07,0.07,0.14,0.07,0.14,Matches
4,5,Charles Abi,fr FRA,FW,Saint-Étienne,fr Ligue 1,22-015,2000,1,1,...,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,2845,Martín Zubimendi,es ESP,MF,Real Sociedad,es La Liga,23-084,1999,31,21,...,1.2,1.2,0.8,2.1,0.05,0.04,0.09,0.05,0.09,Matches
2958,2846,Szymon Żurkowski,pl POL,MF,Empoli,it Serie A,24-214,1997,31,25,...,3.9,3.9,1.2,5.1,0.17,0.06,0.23,0.17,0.23,Matches
2959,2847,Martin Ødegaard,no NOR,MF,Arsenal,eng Premier League,23-131,1998,31,27,...,4.1,4.1,5.8,9.8,0.16,0.22,0.38,0.16,0.38,Matches
2960,2848,Milan Đurić,ba BIH,FW,Salernitana,it Serie A,31-340,1990,28,18,...,4.6,3.9,2.2,6.0,0.24,0.11,0.35,0.20,0.31,Matches


In [ ]:
def get_budget(x):
    url = 'https://www.fifagamenews.com/fifa-22-career-mode-transfer-budgets/'
    #Changing Fbref's HTML so all tables are redable by bs4
    html_content = requests.get(url).text.replace('<!--', '').replace('-->', '')
    df = pd.read_html(html_content)
    Teams=[]
    for i in df:
        Teams.append(str(i).split('Name: 0, dtype: object'))

    t = ' '.join([str(elem) for elem in Teams])

    t=list(t.split("\\n"))
    #t=pd.DataFrame(t)

    #display(Teams[0])
    tno=[]
    x=x
    a=[]
    b=[]
    c=[]
    for i in t:
            a=x.split(" ")
            for j in a:
                j=j.capitalize()
                b.append(j)
            c.append(' '.join([str(elem) for elem in b]))
            break
    bud=[]      
    budget=[]
    for i in t:
        if c[0].lower()==x:
            if c[0] in i:
                i=i.split(c[0])
                x=i[1].split("  ")
                for i in range(1,len(x)):
                        p=x[i].split(" ")
                        if p[0]==" ":
                            bud.append(p[1:])
                            break
                        else:
                            bud.append(p[0:])
                            break
    budget.append('£')
    budget.append('.'.join([str(elem) for elem in bud[0]]))
    budget.append('m')
    return("".join([str(elem) for elem in budget]))


In [ ]:
# Define the base time-series chart.
def get_chart(data):
    hover = alt.selection_single(
        fields=["Value"],
        nearest=True,
        on="mouseover",
        empty="none",
    )

    lines = (
        alt.Chart(data, title="Relation between Value and Age")
        .mark_line()
        .encode(
            x="Value",
            y="Age",
            color="Area",
        )
    )

    # Draw points on the line, and highlight based on selection
    points = lines.transform_filter(hover).mark_circle(size=65)

    # Draw a rule at the location of the selection
    tooltips = (
        alt.Chart(data)
        .mark_rule()
        .encode(
            x="Value",
            y="Age",
            opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
            tooltip=[
                alt.Tooltip("Value", title="Value (GBP)"),
                alt.Tooltip("Age", title="Age"),
            ],
        )
        .add_selection(hover)
    )
    return (lines + points + tooltips).interactive()





In [ ]:
st.title("\t\t\t\t Welcome to The Football Scout Bot!     ")
st.title("Players will be shown from the following datasets:\ni)Out of contract \n ii)Free Agents\n iii)All other players in the top 5 Leagues (England,Spain,France,Germany,Italy)")    
if st.checkbox('Show Out of contract players',key='ooc'):
    st.subheader("Players Going Out Of Contract Names, Details and Statistics:")
    st.write(ooc)
if st.checkbox('Show free agents',key='fa'):
    st.subheader("Players Available For Free:")
    st.title("Players Available For Free")
    st.write(fa)
if st.checkbox("All others:"):
    st.title("Other players")
    st.write(other)

In [67]:
st.title("But first, some observations!")
og=ooc.groupby(['Area']).mean()
of=fa.groupby(['Area']).mean()
st.write(og.iloc[: , -3:])
st.write(of.iloc[: , -3:])
st.write("We can see some obvious information such as Attackers have more npXg and Xg than defenders \n but something interesting to note is that midfielders play more minutes no matter what the age ")

st.subheader("For 'out of contracts dataset':")
st.area_chart(og)


st.subheader("For 'free agents dataset':")
st.area_chart(of)

In [ ]:
chart_o = get_chart(ooc)
chart_f= get_chart(fa)
st.write("We can see that the price for defenders and midfielders rises with age while it drops for attackers.")
st.subheader("For 'out of contracts dataset':")
st.altair_chart(chart_o.interactive(), use_container_width=True)
st.subheader("For 'free agents dataset':")
st.altair_chart(chart_f.interactive(), use_container_width=True)

In [71]:
name = st.text_input("Enter Team Name to search players for")
if(st.button('Submit')):
    x=get_budget(name)
    st.subheader('The budget of the selected team is: ')
    st.subheader(x)

In [153]:
ooc.drop(ooc[ooc['Current Team']==name.capitalize()].index, inplace = True)
ooc.insert(loc=0, column='Total Efficiency', value=ooc['Non Penalty Efficiency']-ooc['Performance_CrdY']-ooc['Performance_CrdR'])
ooc=ooc.sort_values(by=['Age','Value','Total Efficiency'],ascending=[True,False,True])
fa.drop(fa[fa['Previous Team']==name.capitalize()].index, inplace = True)
fa.insert(loc=0, column='Total Efficiency', value=fa['Non Penalty Efficiency']-fa['Performance_CrdY']-fa['Performance_CrdR'])
fa=fa.sort_values(by=['Age','Value','Total Efficiency'],ascending=[True,False,True])

In [154]:
ooc=ooc.drop(['Total Efficiency'],axis = 1)
fa=fa.drop(['Total Efficiency'],axis = 1)

In [130]:
st.subheader("Suggested players:")
if st.checkbox('Show Out of contract players',key=3):
    st.subheader("Best Players Going Out Of Contract Names, Details and Statistics:")
    st.write(ooc.loc[:10, :])
if st.checkbox('Show free agents',key=4):
    st.subheader("Best Players Available For Free:")
    st.write(fa.loc[:10, :])

In [1]:
import streamlit as st
import pandas as pd
import subprocess
import sys
import numpy as np
import requests
import time
import altair as alt
import pandas as pd
import streamlit as st
from vega_datasets import data
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options


2022-04-25 10:24:34.914 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
fa=pd.read_excel('fa.xlsx')
ooc=pd.read_excel('ooc.xlsx')
other=pd.read_excel('other.xlsx')
#display(fa)
#display(ooc)
fa.fillna(0)
ooc.fillna(0)


,Player Name,Position,Current Team,League,Age,DOB,Value,Agent,Primary Nationality,Secondary Nationality,...,Expected_npxG+xA,Per 90 Minutes_xG,Per 90 Minutes_xA,Per 90 Minutes_xG+xA,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xA,Non Penalty Efficiency,Penalty Efficiency,90s rank by age,Area
0,Kylian Mbappé,Centre-Forward,Paris SG,Ligue 1,23,"Dec 20, 1998 (23)",€160.00m,Relatives,France,0,...,129.4,0.84,0.29,1.13,0.77,1.06,2.39,0.909091,17,Attacker
1,Paul Pogba,Central Midfield,Man Utd,Premier League,29,"Mar 15, 1993 (29)",€55.00m,Mino Raiola,France,Guinea,...,36.7,0.26,0.16,0.42,0.18,0.34,0.90,0.615385,32,Midfielder
2,Franck Kessié,Central Midfield,AC Milan,Serie A,25,"Dec 19, 1996 (25)",€45.00m,Ambition Group,Cote d'Ivoire,0,...,31.4,0.23,0.08,0.32,0.11,0.20,0.59,0.851852,31,Midfielder
3,Paulo Dybala,Second Striker,Juventus,Serie A,28,"Nov 15, 1993 (28)",€40.00m,Relatives,Argentina,Italy,...,52.8,0.37,0.20,0.56,0.30,0.50,1.24,0.818182,27,Midfielder
4,Antonio Rüdiger,Centre-Back,Chelsea,Premier League,29,"Mar 3, 1993 (29)",€35.00m,FSB – Spielerberatung GmbH,Germany,Sierra Leone,...,9.9,0.06,0.02,0.08,0.06,0.08,0.16,0.000000,34,Defender
5,Ousmane Dembélé,Right Winger,Barcelona,LaLiga,24,"May 15, 1997 (24)",€30.00m,Relatives,France,Mauritania,...,35.7,0.24,0.32,0.55,0.24,0.55,1.27,1.000000,9,Attacker
6,Andrea Belotti,Centre-Forward,Torino,Serie A,28,"Dec 20, 1993 (28)",€28.00m,0,Italy,0,...,68.6,0.45,0.14,0.59,0.33,0.48,1.20,0.727273,33,Attacker
7,Boubacar Kamara,Defensive Midfield,Marseille,Ligue 1,22,"Nov 23, 1999 (22)",€25.00m,Sport Avenir Management International,France,Senegal,...,6.8,0.03,0.03,0.06,0.03,0.06,0.13,0.000000,13,Midfielder
8,Matthias Ginter,Centre-Back,Bor. M'gladbach,Bundesliga,28,"Jan 19, 1994 (28)",€24.00m,Unique Sports Group,Germany,0,...,17.0,0.08,0.04,0.11,0.08,0.11,0.20,0.000000,41,Defender
9,James Tarkowski,Centre-Back,Burnley,Premier League,29,"Nov 19, 1992 (29)",€22.00m,0,England,Poland,...,15.2,0.06,0.03,0.09,0.06,0.09,0.17,0.000000,43,Defender


In [ ]:
def get_budget(x):
    url = 'https://www.fifagamenews.com/fifa-22-career-mode-transfer-budgets/'
    #Changing Fbref's HTML so all tables are redable by bs4
    html_content = requests.get(url).text.replace('<!--', '').replace('-->', '')
    df = pd.read_html(html_content)
    Teams=[]
    for i in df:
        Teams.append(str(i).split('Name: 0, dtype: object'))

    t = ' '.join([str(elem) for elem in Teams])

    t=list(t.split("\\n"))
    #t=pd.DataFrame(t)

    #display(Teams[0])
    tno=[]
    x=x
    a=[]
    b=[]
    c=[]
    for i in t:
            a=x.split(" ")
            for j in a:
                j=j.capitalize()
                b.append(j)
            c.append(' '.join([str(elem) for elem in b]))
            break
    bud=[]      
    budget=[]
    for i in t:
        if c[0].lower()==x:
            if c[0] in i:
                i=i.split(c[0])
                x=i[1].split("  ")
                for i in range(1,len(x)):
                        p=x[i].split(" ")
                        if p[0]==" ":
                            bud.append(p[1:])
                            break
                        else:
                            bud.append(p[0:])
                            break
    budget.append('£')
    budget.append('.'.join([str(elem) for elem in bud[0]]))
    budget.append('m')
    return("".join([str(elem) for elem in budget]))


In [ ]:
# Define the base time-series chart.
def get_chart(data):
    hover = alt.selection_single(
        fields=["Value"],
        nearest=True,
        on="mouseover",
        empty="none",
    )

    lines = (
        alt.Chart(data, title="Relation between Value and Age")
        .mark_line()
        .encode(
            x="Value",
            y="Age",
            color="Area",
        )
    )

    # Draw points on the line, and highlight based on selection
    points = lines.transform_filter(hover).mark_circle(size=65)

    # Draw a rule at the location of the selection
    tooltips = (
        alt.Chart(data)
        .mark_rule()
        .encode(
            x="Value",
            y="Age",
            opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
            tooltip=[
                alt.Tooltip("Value", title="Value (GBP)"),
                alt.Tooltip("Age", title="Age"),
            ],
        )
        .add_selection(hover)
    )
    return (lines + points + tooltips).interactive()

chart_o = get_chart(ooc)
chart_f= get_chart(fa)
st.write("We can see that the price for defenders and midfielders rises with age while it drops for attackers.")
st.subheader("For 'out of contracts dataset':")
st.altair_chart(chart_o.interactive(), use_container_width=True)
st.subheader("For 'free agents dataset':")
st.altair_chart(chart_f.interactive(), use_container_width=True)



In [ ]:
st.title("\t\t\t\t Welcome to The Football Scout Bot!     ")
st.title("Players will be shown from the following datasets:\ni)Out of contract \n ii)Free Agents\n iii)All other players in the top 5 Leagues (England,Spain,France,Germany,Italy)")    
if st.checkbox('Show Out of contract players',key='ooc'):
    st.subheader("Players Going Out Of Contract Names, Details and Statistics:")
    st.write(ooc)
if st.checkbox('Show free agents',key='fa'):
    st.subheader("Players Available For Free:")
    st.title("Players Available For Free")
    st.write(fa)
if st.checkbox("All others:")
    st.title("Other players")
    st.write(other)

In [67]:
st.title("But first, some observations!")
og=ooc.groupby(['Area']).mean()
of=fa.groupby(['Area']).mean()
st.write(og.iloc[: , -3:])
st.write(of.iloc[: , -3:])
st.write("We can see some obvious information such as Attackers have more npXg and Xg than defenders \n but something interesting to note is that midfielders play more minutes no matter what the age ")

st.subheader("For 'out of contracts dataset':")
st.area_chart(og)


st.subheader("For 'free agents dataset':")
st.area_chart(of)

In [71]:
name = st.text_input("Enter Team Name to search players for")
if(st.button('Submit')):
    x=get_budget(name)
    st.subheader('The budget of the selected team is: ')
    st.subheader(x)
    st.subheader('Stadium Details:')
    st.write('')

In [153]:
ooc.drop(ooc[ooc['Current Team']==name.capitalize()].index, inplace = True)
ooc.insert(loc=0, column='Total Efficiency', value=ooc['Non Penalty Efficiency']-ooc['Performance_CrdY']-ooc['Performance_CrdR'])
ooc=ooc.sort_values(by=['Age','Value','Total Efficiency'],ascending=[True,False,True])
fa.drop(fa[fa['Previous Team']==name.capitalize()].index, inplace = True)
fa.insert(loc=0, column='Total Efficiency', value=fa['Non Penalty Efficiency']-fa['Performance_CrdY']-fa['Performance_CrdR'])
fa=fa.sort_values(by=['Age','Value','Total Efficiency'],ascending=[True,False,True])

In [154]:
ooc=ooc.drop(['Total Efficiency'],axis = 1)
fa=fa.drop(['Total Efficiency'],axis = 1)

In [130]:
st.subheader("Suggested players:")
if st.checkbox('Show Out of contract players',key=3):
    st.subheader("Best Players Going Out Of Contract Names, Details and Statistics:")
    st.write(ooc.loc[:10, :])
if st.checkbox('Show free agents',key=4):
    st.subheader("Best Players Available For Free:")
    st.write(fa.loc[:10, :])